In [1]:
import pickle
from pathlib import Path

pkl_name = Path("/Volumes/CREST Data/David_S_Data/ATI-Data-CanonFatStudy/003/ROIs/ 20220503084651_IQ/high.pkl")

with open(pkl_name, "rb") as f:
    pkl_data = pickle.load(f)

In [34]:
pkl_data.keys()
pkl_data["Phantom Name"]

'20220831121752_IQ.bin'

In [7]:
import pandas as pd

csv_file_path = "/Volumes/CREST Data/David_S_Data/ATI-Data-CanonFatStudy/ROIs/003/20220503084651_IQ/high.csv"

In [8]:
csv_data = pd.read_csv(csv_file_path)
csv_data.head()

,Image Name,Spline x points,Spline y points
0,20220503084651_IQ.bin,"[431, 403, 393, 399, 412, 437, 461]","[243, 238, 222, 198, 187, 186, 214]"


In [33]:
csv_data["Image Name"][0]

'20220503084651_IQ.bin'

In [23]:
import ast

xSpline = ast.literal_eval(csv_data["Spline x points"][0])
ySpline = ast.literal_eval(csv_data["Spline y points"][0])

In [25]:
xSpline.append(xSpline[0])
ySpline.append(ySpline[0])

In [50]:
import scipy.interpolate as interpolate
import numpy as np
import pandas as pd
import ast

def calculateSpline(xpts, ypts):  # 2D spline interpolation
    cv = []
    for i in range(len(xpts)):
        cv.append([xpts[i], ypts[i]])
    cv = np.array(cv)
    if len(xpts) == 2:
        tck, _ = interpolate.splprep(cv.T, s=0.0, k=1)
    elif len(xpts) == 3:
        tck, _ = interpolate.splprep(cv.T, s=0.0, k=2)
    else:
        tck, _ = interpolate.splprep(cv.T, s=0.0, k=3)
    x, y = np.array(interpolate.splev(np.linspace(0, 1, 1000), tck))
    return x, y

preset1_phantom = '20220831121844_IQ.bin'
preset2_phantom = '20220831121752_IQ.bin'

def convertCsvToPkl(csv_file_path: str, preset: int):
    csv_data = pd.read_csv(csv_file_path)
    xSpline = ast.literal_eval(csv_data["Spline x points"][0])
    ySpline = ast.literal_eval(csv_data["Spline y points"][0])
    xSpline.append(xSpline[0])
    ySpline.append(ySpline[0])
    
    interp_x, interp_y = calculateSpline(xSpline, ySpline)
    
    if preset == 2:
        output = {"Image Name": csv_data["Image Name"][0], "Phantom Name": preset2_phantom,
            "Spline X": interp_x, "Spline Y": interp_y}
    else:
        output = {"Image Name": csv_data["Image Name"][0], "Phantom Name": preset1_phantom,
            "Spline X": interp_x, "Spline Y": interp_y}
        
    dest_path = Path(csv_file_path).parent / Path(str(Path(csv_file_path).stem + ".pkl"))
    
    with open(dest_path, mode="wb") as pklfile:
        pickle.dump(output, pklfile, protocol=pickle.HIGHEST_PROTOCOL)


In [61]:
import shutil

roi_folder = Path("/Volumes/CREST Data/David_S_Data/ATI-Data-CanonFatStudy/ROIs/")
file_folder = Path("/Volumes/CREST Data/David_S_Data/ATI-Data-CanonFatStudy")

for patient_folder in roi_folder.iterdir():
    if patient_folder.is_dir() and not patient_folder.name.startswith("."):
        dest_roi_folder = file_folder / patient_folder.name / Path("ROIs")
        dest_roi_folder.mkdir(parents=True, exist_ok=True)
        for scan_folder in patient_folder.iterdir():
            if scan_folder.is_dir() and not scan_folder.name.startswith("."):
                dest_scan_folder = dest_roi_folder / scan_folder.name
                dest_scan_folder.mkdir(parents=True, exist_ok=True)
                for item in scan_folder.iterdir():
                    if item.is_file() and not item.name.startswith(".") and item.name.endswith(".pkl"):
                        shutil.copy(item, dest_scan_folder / item.name)
                
                
        # scan_folder.mkdir(parents=True, exist_ok=True)
        # for item in patient_folder.iterdir():
        #     if item.is_file():
        #         dest = scan_folder / item.name
        #         dest.write_bytes(item.read_bytes())
        # for csv_file in scan_folder.iterdir():
        #     if csv_file.suffix == ".csv" and not csv_file.name.startswith("."):
        #         convertCsvToPkl(csv_file, 2)

In [35]:
interp_x, interp_y = calculateSpline(xSpline, ySpline)

preset1_phantom = '20220831121844_IQ.bin'
preset2_phantom = '20220831121752_IQ.bin'

In [36]:
output = {"Image Name": csv_data["Image Name"][0], "Phantom Name": preset2_phantom,
            "Spline X": interp_x, "Spline Y": interp_y}

In [42]:
dest_path = Path(csv_file_path).parent / Path(str(Path(csv_file_path).stem + ".pkl"))

In [43]:
with open(dest_path, mode="wb") as pklfile:
    pickle.dump(output, pklfile, protocol=pickle.HIGHEST_PROTOCOL)

In [22]:
self.spectralData.splineX, self.spectralData.splineY = calculateSpline(
    self.pointsPlottedX, self.pointsPlottedY
)
self.spectralData.splineX = np.clip(self.spectralData.splineX, a_min=0, a_max=self.spectralData.pixWidth-1)
self.spectralData.splineY = np.clip(self.spectralData.splineY, a_min=0, a_max=self.spectralData.pixDepth-1)

try:
    if self.spectralData.numSamplesDrOut == 1400:
        self.spectralData.splineX = np.clip(self.spectralData.splineX, a_min=148, a_max=573)
        self.spectralData.splineY = np.clip(self.spectralData.splineY, a_min=0.5, a_max=387)
    elif self.spectralData.numSamplesDrOut == 1496:
        self.spectralData.splineX = np.clip(self.spectralData.splineX, a_min=120, a_max=615)
        self.spectralData.splineY = np.clip(self.spectralData.splineY, a_min=0.5, a_max=645)
    # elif self.ImDisplayInfo.numSamplesDrOut != -1:
    #     print("Preset not found!")
    #     return
except (AttributeError, UnboundLocalError):
    pass


[431, 403, 393, 399, 412, 437, 461]